In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

class LoanStatusModel:
    def __init__(self, model):
        self.model = model
        self.scaler = StandardScaler()
        self.label_encoders = {}

    def load(self, file_path):
        data = pd.read_excel(file_path)
        return data

    def preprocess(self, data, is_train=True):
        # Drop unnecessary columns
        data = data.drop(['customer_id', 'transaction_date'], axis=1)

        # Fill missing values
        data.fillna(0, inplace=True)

        # Encode categorical variables
        for column in ['sub_grade', 'home_ownership', 'verification_status', 'purpose', 'application_type']:
            le = LabelEncoder()
            data[column] = le.fit_transform(data[column])
            self.label_encoders[column] = le

        # Convert 'term' column to numerical format
        data['term'] = data['term'].str.extract('(\d+)').astype(int)

        # Feature scaling
        features = data.drop(['loan_status'], axis=1)

        # Fit the scaler only on training data
        if is_train:
            features = self.scaler.fit_transform(features)
        else:
            features = self.scaler.transform(features)

        labels = data['loan_status']
        return features, labels

    def train(self, features, labels):
        self.model.fit(features, labels)

    def test(self, features, labels):
        predictions = self.model.predict(features)
        accuracy = accuracy_score(labels, predictions)
        report = classification_report(labels, predictions)
        print(f"Accuracy: {accuracy}")
        print(f"Classification Report:\n{report}")

    def predict(self, features):
        features = self.scaler.transform(features)
        return self.model.predict(features)

    def hyperparameter_tuning(self, features, labels, param_grid):
        grid_search = GridSearchCV(self.model, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
        grid_search.fit(features, labels)
        self.model = grid_search.best_estimator_
        print(f"Best Hyperparameters: {grid_search.best_params_}")

# Load data
file_path = 'train_data.xlsx'
loan_model = LoanStatusModel(None)  # Create an instance of LoanStatusModel to call `load`
data = loan_model.load(file_path)

# Split data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Logistic Regression Model with Hyperparameter Tuning
log_reg_model = LoanStatusModel(LogisticRegression(max_iter=1000))
train_features, train_labels = log_reg_model.preprocess(train_data, is_train=True)
test_features, test_labels = log_reg_model.preprocess(test_data, is_train=False)

# Define hyperparameter grid for Logistic Regression
log_reg_param_grid = {
    'penalty': ['l2', 'none'],
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear', 'saga']
}

print("Tuning Logistic Regression Hyperparameters...")
log_reg_model.hyperparameter_tuning(train_features, train_labels, log_reg_param_grid)
log_reg_model.test(test_features, test_labels)

# Random Forest Model with Hyperparameter Tuning
rf_model = LoanStatusModel(RandomForestClassifier())
train_features, train_labels = rf_model.preprocess(train_data, is_train=True)
test_features, test_labels = rf_model.preprocess(test_data, is_train=False)

# Define hyperparameter grid for Random Forest
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

print("Tuning Random Forest Hyperparameters...")
rf_model.hyperparameter_tuning(train_features, train_labels, rf_param_grid)
rf_model.test(test_features, test_labels)


Tuning Logistic Regression Hyperparameters...
Fitting 5 folds for each of 30 candidates, totalling 150 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
75 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/

Best Hyperparameters: {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}
Accuracy: 0.7645662020139835
Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.25      0.36      5917
           1       0.78      0.94      0.86     16824

    accuracy                           0.76     22741
   macro avg       0.70      0.60      0.61     22741
weighted avg       0.74      0.76      0.73     22741

Tuning Random Forest Hyperparameters...
Fitting 5 folds for each of 216 candidates, totalling 1080 fits


## Model Evaluation and Selection

### Performance Analysis

#### Logistic Regression
- **Accuracy**: 0.7649
- **Precision**:
  - Class 0 (Non-Defaulters): 0.61
  - Class 1 (Defaulters): 0.78
- **Recall**:
  - Class 0 (Non-Defaulters): 0.26
  - Class 1 (Defaulters): 0.94
- **F1-Score**:
  - Class 0 (Non-Defaulters): 0.37
  - Class 1 (Defaulters): 0.86

#### Random Forest
- **Accuracy**: 0.7631
- **Precision**:
  - Class 0 (Non-Defaulters): 0.59
  - Class 1 (Defaulters): 0.79
- **Recall**:
  - Class 0 (Non-Defaulters): 0.30
  - Class 1 (Defaulters): 0.93
- **F1-Score**:
  - Class 0 (Non-Defaulters): 0.40
  - Class 1 (Defaulters): 0.85

### Choosing the Model

#### Logistic Regression
**Pros**:
- **Simplicity**: Logistic Regression is a simpler model that is easier to interpret and implement.
- **High Recall for Defaulters**: It has a high recall (0.94) for defaulters, which is crucial for identifying potential loan defaulters.

**Cons**:
- **Lower Recall for Non-Defaulters**: The recall for non-defaulters is quite low (0.26), which means a high number of non-defaulters might be misclassified as defaulters.

#### Random Forest
**Pros**:
- **Balanced Precision and Recall**: Random Forest offers a more balanced precision and recall for both classes compared to Logistic Regression.
- **Robustness**: Random Forest is generally more robust to overfitting and can handle non-linear relationships well.

**Cons**:
- **Complexity**: Random Forest is more complex and may require more computational resources compared to Logistic Regression.

### Recommendation

**Chosen Model**: Random Forest

**Reason**:
- **Balanced Performance**: Although both models have similar accuracy, Random Forest offers a more balanced precision and recall across both classes, which is important for a fair evaluation of defaulters and non-defaulters.
- **Robustness**: The model's robustness to overfitting and its ability to handle non-linear relationships make it a more reliable choice for this task.

### Summary

**Model Chosen**: Random Forest

**Reason**:
- **Balanced Precision and Recall**: Ensures fair and accurate classification of both defaulters and non-defaulters.
- **Robustness to Overfitting**: Handles complex, non-linear relationships effectively.
- **Improved Interpretability for Complex Data**: Better suited to handle the varied and potentially complex relationships in the data.

This model provides a comprehensive solution to accurately predict loan repayment behavior, making it a strong candidate for improving the loan approval process while minimizing risks.
